In [ ]:
import torch
import transformers
from torch import cuda, bfloat16


device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device

In [ ]:
model_name = 'REELICIT/Llama-2-7b-chat-hf-ReqBrain'

model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, use_fast = False)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.eval()
model.to(device)
print(f"Model loaded on {device}")

In [ ]:
___inst = tokenizer.convert_ids_to_tokens(tokenizer("[/INST]")["input_ids"])[1:]
___java = tokenizer.convert_ids_to_tokens(tokenizer("```java")["input_ids"])#[1:]
___end_of_ = tokenizer.convert_ids_to_tokens(tokenizer("/end_of_")["input_ids"])[1:]
___user = tokenizer.convert_ids_to_tokens(tokenizer("[/user]")["input_ids"])[1:]
___inst_small = tokenizer.convert_ids_to_tokens(tokenizer("[/Inst")["input_ids"])[1:]
___hash_tag = tokenizer.convert_ids_to_tokens(tokenizer("#")["input_ids"])[1:]
___star = tokenizer.convert_ids_to_tokens(tokenizer("*")["input_ids"])[1:]

stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [___inst, ___inst_small, [tokenizer.eos_token], ___end_of_, ___user, ['```'], ___hash_tag, ___star]
]

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
# stopping_criteria(
#     torch.LongTensor([tokenizer.convert_tokens_to_ids(_)]).to(device),
#     torch.FloatTensor([0.0])
# )

In [ ]:
pipe = transformers.pipeline(
    model = model,
    tokenizer = tokenizer,
    return_full_text = True, # Set it to True when combining with LangChain
    task='text-generation',
    device=device,
    stopping_criteria = stopping_criteria,  
    temperature = 0.1,
    top_p = 0.15,  
    top_k = 0,  
    max_new_tokens = 512,  
    repetition_penalty = 1.3
)

In [ ]:
import datasets

evaluation_set = datasets.load_from_disk("./models_prediction_dataset")
evaluation_set

In [ ]:
from tqdm import tqdm

completion = []

for query in tqdm(evaluation_set['query']):
    result = result = pipe(f"<s>[INST] {query} [/INST]")
    result = result[0]['generated_text'].split('[/INST]')[-1].strip("[/INST]")
    result = result.strip("```java")
    result = result.strip("/end_of_")
    result = result.strip(" ")
    result = result.strip("[/user]")
    result = result.strip("[/Inst")
    result = result.strip("```")
    result = result.strip("#")
    result = result.strip("*")
    result = result.strip(" ")
    completion.append(result)

In [ ]:
evaluation_set = evaluation_set.add_column("llama2_7b_chat_hf_preds", completion)

In [ ]:
evaluation_set.save_to_disk("./models_prediction_dataset")